In [93]:
from datetime import datetime
import pandas as pd
import numpy as np

In [96]:
f = open("fvtt-log-Thu-Feb-17-2022.txt",encoding="utf8")
data = f.read()
data_list = data.split("---------------------------")
clean = []
characters = []
#df = pd.DataFrame(columns= ["Character", "Roll Result","Action","D20 Roll", "Date"])
try:
    for i, item in enumerate(data_list):
        temp_list = item.split("\n")
        temp_list = list(filter(None, temp_list))
        roll_result = temp_list[1]
        try:
            found_action = temp_list[2].find("<strong>")
        except:
            found_action = -1
        if roll_result.isnumeric() and found_action>-1:
            dateString = temp_list[0][1:temp_list[0].index(",")]
            date = datetime.strptime(dateString,'%m/%d/%Y')
            character = temp_list[0][temp_list[0].index("]")+2:]
            if character not in characters:
                characters.append(character)
            action = temp_list[2][temp_list[2].index("<strong>")+8:temp_list[2].index("</strong>")]
            d20roll = temp_list[-1][temp_list[-1].index("=")+1:temp_list[-1].index("=")+4].strip()
            clean.append([character,roll_result,action,d20roll,date])
        data_list[i] = temp_list
    df = pd.DataFrame(clean, columns= ["Character", "Roll Result","Action","D20 Roll", "Date"])
except Exception as e:
    print(dateString)
    print(date)
    print(data_list[i])
    print("error in line " + str(i))
    print(e)


In [97]:
df

,Character,Roll Result,Action,D20 Roll,Date
0,Nemmia Bramblecloak,16,Strike: sickle,7,2021-07-06
1,Npc Actor,17,Saving Throw: Reflex,17,2021-07-06
2,Npc Actor,10,Saving Throw: Reflex,10,2021-07-06
3,Abberton Ruffians,13,Saving Throw: Reflex,8,2021-07-06
4,Nemmia Bramblecloak,14,Strike: sickle,5,2021-07-06
...,...,...,...,...,...
4535,Redcap,17,Strike: Scythe,7,2022-02-13
4536,Redcap,19,Strike: Scythe,4,2022-02-13
4537,Redcap,27,Strike: Scythe,17,2022-02-13
4538,Melinoë,16,Primal Spell Attack,1,2022-02-13


In [ ]:
data_list

In [100]:
import matplotlib.pyplot as plt
import statistics

In [ ]:
players = ["Lucius","Trujh Spellshot","Mr. Biscuit Oliva","Razonixx","Ylem"]
summary = []
sinceWhen = datetime(2022,2,10)
x_dice = []

for player in characters:
    historic = []
    x_dice = []
    rolls = pd.DataFrame(data ={"Rolls":[0]*20})
    rolls.index = rolls.index+1
    #xi = list(range(len(rolls.index)))
    newdf = df[df["Character"] == player]
    newdf = newdf[newdf["Date"] > sinceWhen]
    newdf["D20 Roll"] = pd.to_numeric(newdf["D20 Roll"])
    if len(newdf.index) > 0:
        plt.figure()
        counts = newdf['D20 Roll'].value_counts()
        counts = newdf['D20 Roll'].value_counts()
        counts = counts.to_frame()
        counts.index = pd.to_numeric(counts.index)
        counts = counts.sort_index()
        newRolls = pd.concat([rolls, counts],axis=1)
        newRolls = newRolls.fillna(value={"D20 Roll":0})
        newRolls['D20 Roll'].plot.bar()
        plt.xlabel('Roll')
        plt.title(player + " Mean: " + str(newdf['D20 Roll'].mean()))
        plt.savefig(player + ".jpg", dpi=72)
        summary.append([newdf['D20 Roll'].mean(),player,len(newdf.index)])   

        
summary.sort()

In [ ]:

players = ["Lucius","Trujh Spellshot","Mr. Biscuit Oliva","Razonixx","Ylem"]
summary = []
sinceWhen = datetime(2022,1,1)
for player in characters:
    rolls=[0] * 20
    historic = []
    x_dice = []
    for i in range(20):
        x_dice.append(i+1)
    
    newdf = df[df["Character"] == player]
    newdf = newdf[newdf["Date"] > sinceWhen]
    for attempt in clean:
        if attempt[0] == player and attempt[4]>sinceWhen:
            roll = int(attempt[3])
            rolls[roll-1]+=1
            historic.append(roll)
    if len(historic) > 0:
        fig, axs = plt.subplots(1, 1, figsize=(9, 3), sharey=True)
        summary.append([statistics.mean(historic),player,len(historic)])
        plt.bar(x_dice,rolls)
        plt.text(1,max(rolls)+1,player + "  Mean = " + str(statistics.mean(historic)))
        plt.text(1,max(rolls)+.5,"In " + str(len(historic)) +" rolls")
summary.sort()

In [104]:
summary.reverse()
summary

[[14.3, 'Stegosaurus', 10],
 [11.172413793103448, 'Razonixx', 29],
 [10.9375, 'Gluttondark Babau', 16],
 [10.568181818181818, 'Redcap', 88],
 [10.5, 'Xulgath Demon-Caller', 2],
 [10.363636363636363, 'Clay Golem', 11],
 [10.16, 'Trujh Spellshot', 25],
 [10.05, 'Melinoë', 20],
 [9.421052631578947, 'Mr. Biscuit Oliva', 19],
 [9.043478260869565, 'Lucius', 23],
 [9.0, 'Xulgath Mage', 1],
 [4.0, 'Corvian', 1]]